1) Необходимо собрать информацию о вакансиях на должность программиста или разработчика с сайта job.ru или hh.ru. (Можно с обоих сразу) Приложение должно анализировать несколько страниц сайта. Получившийся список должен содержать в себе:

*Наименование вакансии, 
*Предлагаемую зарплату 
*Ссылку на саму вакансию  
2) Доработать приложение таким образом, чтобы можно было искать разработчиков на разные языки программирования (Например Python, Java, C++)

In [ ]:
from requests import Request, Session, RequestException, Response
from lxml import html
import urllib.parse

MAX_PAGE = 5  # ограничение количества страниц

In [ ]:
# Возвращает результат запроса
def html_from_url(session, url, params, timeout=50):
    msg = ''
    response = Response()
    try:
        headers = {'accept': '*/&*',
                   'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                                 '(KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36'}
        request = Request('GET', url, params=params, headers=headers)
        prepare_request = session.prepare_request(request)

        response = session.send(prepare_request,
                                timeout=timeout
                                )
        # print(response.url)
    except RequestException as e:
        msg = str(e)
    return response.text, response.status_code, msg


# Возвращает параметры ссылки на следующую страницу
def hh_next_url(html_text):
    root = html.fromstring(html_text)
    results_list = root.xpath('//a[contains(@class, "bloko-button HH-Pager-Controls-Next HH-Pager-Control")]')
    if len(results_list) > 0:
        # print(results_list[0].get('href'))
        return params_from_url(results_list[0].get('href'))
    else:
        return None  # Если ссылки на следующую страницу нет, то это последняя страница


# Разбирает параметры ссылки
def params_from_url(url):
    params = {}
    if len(url) > 1:
        for spam in url.split('?')[1].split('&'):
            key = spam.split('=')[0]
            value = spam.split('=')[1]
            params[key] = urllib.parse.unquote(value)
        return params
    else:
        return None


# парсинг страницы с вакансиями
def hh_parse_vacancy(html_text):
    result = []
    root = html.fromstring(html_text)
    vacancies_list = root.xpath('//a[contains(@class, "bloko-link HH-LinkModifier")]')
    salary_list = root.xpath('//div[contains(@class, "vacancy-serp-item__sidebar")]')
    for text_vacancy, salary_vacancy in zip(vacancies_list, salary_list):
        result.append({
            'title': text_vacancy.text_content(),
            'url': text_vacancy.get('href').split('?')[0],
            'salary': salary_vacancy.text_content() if len(salary_vacancy.text_content()) > 0 else ""
        })
    return result


# печать вакансий по заданным параметрым
def print_hh_vacancies(hh_vacancies, vacancy, area, limit=10):
    print('*' * 120)
    print(
        f'По заданным параметрам \"{vacancy} {area}\" всего найдено {len(hh_vacancies)}, в том числе были найдены следуюшие вакансии')
    i = 1
    for vacancy in hh_vacancies[:limit]:
        salary_text = ''
        if len(vacancy["salary"]) > 0:
            salary_text = f'с зарплатой {vacancy["salary"]}'
        else:
            salary_text = 'зарплата не указана'
        print(f'{i}.Вакансия {vacancy["title"]} по ссылке {vacancy["url"]} {salary_text}')
        i += 1


# основная функция сбора данных о вакансиях по заданным параметрам
def hh_get_vacancy_with_params(hh_session, hh_url, vacancy, area, max_page=MAX_PAGE):
    hh_params = {'text': f'{vacancy} {area}'}
    hh_vacancies = []
    page = 0
    while hh_params != None and page < MAX_PAGE:
        hh_html, status_code, msg = html_from_url(hh_session, hh_url, hh_params)
        if status_code == 200:
            hh_vacancies.extend(hh_parse_vacancy(hh_html))
            hh_params = hh_next_url(hh_html)
        else:
            print(f'Ошибка загрузки страницы {hh_url}: {msg}')
            break
        page += 1
    return hh_vacancies

In [ ]:
vacancy_list = ['Python', 'Java', 'C++']
area = 'Россия'

hh_url = 'https://hh.ru/search/vacancy?'
hh_session = Session()
hh_vacancies = {}
for spam in vacancy_list:
    eggs = f'Программист {spam}'
    hh_vacancies[eggs] = hh_get_vacancy_with_params(hh_session, hh_url, eggs, area, max_page=MAX_PAGE)
    print_hh_vacancies(hh_vacancies[eggs], eggs, area, limit=10)